# Create Simulation



In [2]:
from IPython.parallel import Client


ModuleNotFoundError: No module named 'parallel'

In [3]:

rc = Client()
dv = rc[:]


@dv.remote(block = True)
def GetSimulationResults():
    # Most Basic
    import matplotlib.pyplot as plt
    import matplotlib as mpl
    import seaborn as sns
    import pandas as pd
    import numpy as np
    import math
    from os import path
    import sys
    from datetime import datetime
    from scipy.stats import norm
    import scipy.stats as ss
    import copy
    import warnings
    warnings.simplefilter('error')
    import datetime

    # Filter Specific Functionalities
    from filterpy.kalman import UnscentedKalmanFilter
    from filterpy.common import Q_discrete_white_noise
    from filterpy.kalman import MerweScaledSigmaPoints

    # Gradient Based
    import autograd.numpy as a_np
    import autograd.scipy.stats as a_ss
    from autograd import grad, multigrad_dict, elementwise_grad, jacobian

    # Self Defined functions5
    from DataMaker_New import dataCleaner_new
    from ESN import EchoState
    from Loss_diag_updateV import loss
    from GenerateSimulatedData import make_simulated_data

    u_dim = 8
    topK = 10
    n_scenario = 1
    np.random.seed(1234)
    train_list, valid_list, test_list, true_vol = make_simulated_data(n_scenarios= n_scenario, n_steps=200 ,true_vol_std=0.01, topK=topK, init_price=2000, u_dim=u_dim)

    train_dates = list(train_list[0].keys())
    valid_dates = list(valid_list[0].keys())
    test_dates = list(test_list[0].keys())
    num_train_day = max(train_dates)-min(train_dates)+1
    num_valid_day = max(valid_dates)-min(valid_dates)+1
    num_test_day = max(test_dates)-min(test_dates)+1
    num_train_obs = num_train_day*topK 
    num_valid_obs = num_valid_day*topK
    num_test_obs = num_test_day*topK 

    print("Attached {} train days; {} train observations".format(num_train_day, num_train_obs))
    print("Attached {} valid days; {} valid observations".format(num_valid_day, num_valid_obs))
    print("Attached {} test days; {} test observations".format(num_test_day, num_test_obs))

    # Define transition Dynamics
    # Positional requirement, no real effect, will be replaced by ESN and BS defined further down
    def fx(x, dt):
        return x*dt
    def hx(x):
        return x

    def sigmoid(x):
        return np.divide(1, 1+np.exp(-x))


    def ESNf(theta_now, dt, **fx_args):
        """
        :params theta_now theta_t

        **fx_args
        :params G reservoir transition matrix
        :params G_in input translation matrix
        :params b reservoir bias vector
        :params u_quad_now u_t^2
        """
        G = fx_args["G"]
        G_in = fx_args["G_in"]
        b = fx_args["b"]
        u_quad_now = fx_args["u_quad_now"]
        theta_next = sigmoid(np.matmul(G, theta_now) + np.matmul(G_in, u_quad_now) + b)
        return theta_next


    def BSf(theta_t, **hx_args):
        """
        :params theta_t current state vector

        **hx_args
        :params p_t current asset price
        :params r_t current asset price
        :params K_t strike price, 1d vector
        :params T_t maturity time, 1d vector
        """
        p_t = hx_args["p_t"]
        r_t = hx_args["r_t"]
        K_t = hx_args["K_t"]
        T_t = hx_args["T_t"]

        volatility_std = np.average(theta_t)
        volatility_var = np.power(volatility_std, 2)

        dividor = np.sqrt(volatility_var * T_t)
        d_pls = (np.log(p_t/K_t) + np.multiply(r_t+volatility_var/2, T_t)) / dividor
        d_mns = (np.log(p_t/K_t) + np.multiply(r_t-volatility_var/2, T_t)) / dividor
        y_t = np.multiply(p_t, norm.cdf(d_pls)) - np.multiply(np.multiply(K_t, np.exp(-r_t*T_t)), norm.cdf(d_mns))
        return y_t

    def is_pos_def(x):
        return np.all(np.linalg.eigvals(x) > 0)

    def getCoverage(mean, std, truth):
        check_points = np.arange(5, 100, 5) / 100


        samples = np.random.normal( mean, std, (50, 100) )

        coverage_for_1_observation = []
        for check_point in check_points:
            lower = np.quantile(samples, 0.5-check_point/2, axis=1 )
            upper = np.quantile(samples, 0.5+check_point/2, axis=1 ) 
            coverage_for_1_observation.append( float(np.count_nonzero( np.logical_and( lower < truth , truth < upper ) ) / 50 ) )
        return coverage_for_1_observation

    np.random.seed(1234)
    gradient = grad(loss)


    forward_k = 20

    error_for_all_scenario = []
    best_itr_for_all_scenario = []
    best_train_itr_for_all_scenario = []
    best_valid_itr_for_all_scenario= []
    params_for_all_scenario = []
    ms_s_for_all_scenario = []
    Cs_s_for_all_scenario = []
    pred_Cs_for_all_scenario = []

    train_pred_ms_for_all_scenario = []
    pred_ms_for_all_scenario = []


    theta_dim = 8
    print("theta_dim = {}".format(theta_dim))   

    for scenario_idx in range(n_scenario):
        best_valid_error = 999
        best_valid_itr = 0

        train_data = train_list[scenario_idx]
        valid_data = valid_list[scenario_idx]
        test_data = test_list[scenario_idx]

        step = 0.03
        m_t_1 = [np.array([0]), np.array([0]), np.array([0]), np.array([0]), np.array([0])]
        v_t_1 = [np.array([0]), np.array([0]), np.array([0]), np.array([0]), np.array([0])]
        beta_1 = 0.9
        beta_2 = 0.999
        epsillon = 1e-8

        print("The {}th scenario".format(scenario_idx+1))
        alpha = 0.005
        connectivity = 0.4
        spectral_radius = 1.2
        bday_p_year = 252
        bias = -2.51

        v_scale = 4.0
        w = 0.0001

        # initialize ESN dynamics
        ESN = EchoState(theta_dim, u_dim, connectivity, spectral_radius, bias)
        ESN.G_in *= 2
        v = v_scale

        last_avg_error = 100
        best_itr = 0
        best_train_error = 100
        best_train_itr = 0

        #%%%%%% Error trajectories

        test_error_traj = []
        params_traj = []
        ms_s_traj = []
        Cs_s_traj = []

        train_pred_ms_traj = []
        pred_ms_traj = []
        pred_Cs_traj = []

        for em_itr in range(25):
            dt=1
            points = MerweScaledSigmaPoints(theta_dim, alpha=.001, beta=2., kappa=0)
            ukf = UnscentedKalmanFilter(dim_x=ESN.theta_dim, dim_z=topK, dt=1, fx=fx, hx=hx, points=points)
            ukf.x = np.ones(ESN.theta_dim)* 0.006 # initial state
            ukf.P *= 0.01 # initial uncertainty
            ukf.R = np.diag([v]*topK) # 1 standard
            ukf.Q = np.eye(theta_dim)*w

            ms = [ukf.x.copy()]
            Cs = [ukf.P.copy()]
            u_quad_s = []
            ps = []
            rs = []
            ys = []
            Ks = []
            us = []
            Ts = []

            all_days = sorted(list(train_data.keys()))
            for day in all_days:
                # get observable data
                ## Stock Wise and market wise data
                r_t = train_data[day]["risk ir"] 
                u_t = train_data[day]["return"].reshape([-1]) * math.sqrt(bday_p_year)
                p_t = train_data[day]["price"]

                ## Option wise data
                y_t = train_data[day]["option price"]
                K_t = train_data[day]["Strike"]
                T_t = train_data[day]["Exercise Time"]

                ukf.predict(fx = ESNf, G=ESN.G, G_in=ESN.G_in, b= ESN.b, u_quad_now=np.power(u_t,2))
                ukf.update(y_t, hx=BSf, p_t=p_t, r_t=r_t, K_t=K_t, T_t=T_t)
                ms.append(ukf.x_post.copy())
                Cs.append(ukf.P_post.copy())
                u_quad_s.append(np.power(u_t,2))
                us.append(u_t)
                ps.append(p_t)
                rs.append(r_t)
                ys.append(y_t)
                Ks.append(K_t)
                Ts.append(T_t)

            ms = np.array(ms)
            Cs = np.array(Cs)
            ms_s, Cs_s, _, cvs_s = ukf.rts_smoother(ms, Cs, fx=ESNf, G=ESN.G, G_in=ESN.G_in, b=ESN.b, u_s=us)


            params = [copy.copy(ESN.G), copy.copy(ESN.G_in), copy.copy(ESN.b), copy.copy(w), copy.copy(v)]


            try:
                curr_loss = loss(params,
                                 ms_s, Cs_s, cvs_s,
                                 u_quad_s,
                                 ps, rs, Ks, Ts, ys,
                                 num_train_obs,
                                 reg=alpha)
                curr_grad = gradient(params,
                                     ms_s, Cs_s, cvs_s,
                                     u_quad_s,
                                     ps, rs, Ks, Ts, ys,
                                     num_train_obs,
                                     reg=alpha)

    ######################## Adam Optimizer ###########################

                m_t = [None, None, None, None, None]
                v_t = [None, None, None, None, None]
                m_t_hat = [None, None, None, None, None]
                v_t_hat = [None, None, None, None, None]

                m_t[0] = beta_1 * m_t_1[0] + (1-beta_1) * curr_grad[0]
                v_t[0] = beta_2 * v_t_1[0] + (1-beta_2) * (curr_grad[0] * curr_grad[0])
                m_t_hat[0] = m_t[0] / (1-beta_1 ** (em_itr+1))
                v_t_hat[0] = v_t[0] / (1-beta_2 ** (em_itr+1))
                params[0] -= step * m_t_hat[0] / (np.sqrt(v_t_hat[0]) + epsillon)

                m_t[1] = beta_1 * m_t_1[1] + (1-beta_1) * curr_grad[1]
                v_t[1] = beta_2 * v_t_1[1] + (1-beta_2) * (curr_grad[1] * curr_grad[1])
                m_t_hat[1] = m_t[1] / (1-beta_1 ** (em_itr+1))
                v_t_hat[1] = v_t[1] / (1-beta_2 ** (em_itr+1))
                params[1] -= step * m_t_hat[1] / (np.sqrt(v_t_hat[1]) + epsillon)

                m_t[2] = beta_1 * m_t_1[2] + (1-beta_1) * curr_grad[2]
                v_t[2] = beta_2 * v_t_1[2] + (1-beta_2) * (curr_grad[2] * curr_grad[2])
                m_t_hat[2] = m_t[2] / (1-beta_1 ** (em_itr+1))
                v_t_hat[2] = v_t[2] / (1-beta_2 ** (em_itr+1))
                params[2] -= step * m_t_hat[2] / (np.sqrt(v_t_hat[2]) + epsillon)

                m_t[3] = beta_1 * m_t_1[3] + (1-beta_1) * curr_grad[3]
                v_t[3] = beta_2 * v_t_1[3] + (1-beta_2) * (curr_grad[3] * curr_grad[3])
                m_t_hat[3] = m_t[3] / (1-beta_1 ** (em_itr+1))
                v_t_hat[3] = v_t[3] / (1-beta_2 ** (em_itr+1))
                params[3] -= step * m_t_hat[3] / (np.sqrt(v_t_hat[3]) + epsillon)

                m_t[4] = beta_1 * m_t_1[4] + (1-beta_1) * curr_grad[4]
                v_t[4] = beta_2 * v_t_1[4] + (1-beta_2) * (curr_grad[4] * curr_grad[4])
                m_t_hat[4] = m_t[4] / (1-beta_1 ** (em_itr+1))
                v_t_hat[4] = v_t[4] / (1-beta_2 ** (em_itr+1))
                params[4] -= step * m_t_hat[4] / (np.sqrt(v_t_hat[4]) + epsillon)

                m_t = m_t_1
                v_t = v_t_1

    ###################################################################

            except RuntimeWarning or LinAlgError or OverflowError or w <= 0:
                print("set w to 1e-6 ensure positive")
                pass

            ESN.G, ESN.G_in, ESN.b, _, __ = params
            w = max(0.000001, params[3][0]) # Change form arraybox back to scalar
            v = max(0.000001, params[4][0]) # Change form arraybox back to scalar


    ################################################################### Training Performance

            train_pred_ms = []
            train_errors = []
            np.delete(ms_s,0,0)
            np.delete(Cs_s,0,0)
            for i in range(0, len(ms_s)-1):

                ukf.predict(fx = ESNf,  G=ESN.G, G_in=ESN.G_in, b= ESN.b, u_quad_now=u_quad_s[i])
                train_pred_ms.append(np.average(ukf.x_prior))
                pred_price = BSf(ukf.x_prior, p_t=ps[i], r_t=rs[i], K_t=Ks[i], T_t=Ts[i])
                error = np.average(np.abs(ys[i]-pred_price) /ys[i])
                train_errors.append(error)
                ukf.update(ys[i], hx=BSf, p_t=ps[i], r_t=rs[i], K_t=Ks[i], T_t=Ts[i])

            train_pred_ms_traj.append(train_pred_ms)
            avg_train_error = np.average(train_errors)
    ################################################################### Validation Performance
            ukf.x = ms_s[-1]
            ukf.P = Cs_s[-1]
            valid_errors = []
            pred_ms = []
            pred_Cs = []

            all_valid_days = sorted(list(valid_data.keys()))
            for day in all_valid_days:
                # get observable data
                ## Stock Wise and market wise data
                r_t = valid_data[day]["risk ir"]
                u_t = valid_data[day]["return"].reshape([-1]) * math.sqrt(bday_p_year)
                p_t = valid_data[day]["price"]

                ## Option wise data
                y_t = valid_data[day]["option price"]
                K_t = valid_data[day]["Strike"]
                T_t = valid_data[day]["Exercise Time"]

                ukf.predict(fx = ESNf, G=ESN.G, G_in=ESN.G_in, b= ESN.b, u_quad_now=np.power(u_t,2))
                pred_ms.append(ukf.x_prior)
                pred_Cs.append(ukf.P_prior)
                pred_price = BSf(ukf.x_prior, p_t=p_t, r_t=r_t, K_t=K_t, T_t=T_t)
                error = np.average(np.abs(y_t-pred_price) /y_t)
                valid_errors.append(error)

                ukf.update(y_t, hx=BSf, p_t=p_t, r_t=r_t, K_t=K_t, T_t=T_t)

            avg_valid_error = np.average(valid_errors) 

    ################################################################### Testing Performance

            # forward K step prediction
            k_step_test_error = []

            test_k_step_pred_ms = {}
            test_k_step_pred_Cs = {}

            for k in range(1, min(forward_k+1, num_test_day+1)): 

                test_k_step_pred_ms[k] = []
                test_k_step_pred_Cs[k] = []

                test_errors = []            
                all_test_days = sorted(list(test_data.keys()))

                for idx in range(len(all_test_days) - k):

                    ukf_pred = copy.deepcopy(ukf)

                    # For loop for k predictions forward
                    for shift in range(k) :
                        # get observable data
                        ## Stock Wise and market wise data
                        r_t = test_data[all_test_days[idx+shift]]["risk ir"]
                        u_t = test_data[all_test_days[idx+shift]]["return"].reshape([-1]) * math.sqrt(bday_p_year)
                        p_t = test_data[all_test_days[idx+shift]]["price"]

                        ## Option wise data
                        y_t = test_data[all_test_days[idx+shift]]["option price"]
                        K_t = test_data[all_test_days[idx+shift]]["Strike"]
                        T_t = test_data[all_test_days[idx+shift]]["Exercise Time"]

                        ukf_pred.predict(fx = ESNf, G=ESN.G, G_in=ESN.G_in, b= ESN.b, u_quad_now=np.power(u_t,2))


                    test_k_step_pred_ms[k].append(ukf_pred.x_prior)
                    test_k_step_pred_Cs[k].append(ukf_pred.P_prior)


                    pred_price = BSf(ukf_pred.x_prior, p_t=p_t, r_t=r_t, K_t=K_t, T_t=T_t)
                    error = np.average(np.abs(y_t-pred_price) /y_t)
                    test_errors.append(error)


                    # shift base one step forward
                    r_t = test_data[all_test_days[idx]]["risk ir"] 
                    u_t = test_data[all_test_days[idx]]["return"].reshape([-1]) * math.sqrt(bday_p_year)
                    p_t = test_data[all_test_days[idx]]["price"]

                    ## Option wise data
                    y_t = test_data[all_test_days[idx]]["option price"]
                    K_t = test_data[all_test_days[idx]]["Strike"]
                    T_t = test_data[all_test_days[idx]]["Exercise Time"]

                    ukf.predict(fx = ESNf, G=ESN.G, G_in=ESN.G_in, b= ESN.b, u_quad_now=np.power(u_t,2))
                    ukf.update(y_t, hx=BSf, p_t=p_t, r_t=r_t, K_t=K_t, T_t=T_t)

                if test_errors != []:
                    avg_test_error = np.average(test_errors)
                    k_step_test_error.append(avg_test_error)
                else:
                    pass


    ################################################################### Show details
            print("EM Iter {} | Loss {:.4f} | Train Error {:.4f} | Val Error {:.4f} | Test 1-Error {:.4f} | Test max-Error {:.4f} |"  
                  " Dyn var {:.2f} | Obs var {:.2f}".format(em_itr, curr_loss,
                                                            avg_train_error, avg_valid_error, k_step_test_error[0], k_step_test_error[-1],
                                                            w, v))

            # Get best validation itr
            if best_valid_error > avg_valid_error:
                best_valid_error = avg_valid_error
                best_valid_itr = em_itr

            if last_avg_error < avg_valid_error and em_itr-best_itr >= 10:
                pass 
            elif last_avg_error < avg_valid_error and em_itr-best_itr < 10:
                pass
            elif last_avg_error > avg_valid_error and em_itr-best_itr < 10:
                last_avg_error = avg_valid_error
                model_performance = avg_test_error
                best_itr = em_itr

            if best_train_error > avg_train_error and em_itr-best_train_itr < 10:
                best_train_error = avg_train_error
                best_train_itr = em_itr
            elif best_train_error < avg_train_error:
                step = 0.002

            test_error_traj.append(k_step_test_error)
            params_traj.append(copy.deepcopy(params))
            ms_s_traj.append(ms_s)
            Cs_s_traj.append(Cs_s)
            pred_ms_traj.append(test_k_step_pred_ms)
            pred_Cs_traj.append(test_k_step_pred_Cs)

        error_for_all_scenario.append(test_error_traj)
        best_itr_for_all_scenario.append(best_itr)
        best_train_itr_for_all_scenario.append(best_train_itr)
        best_valid_itr_for_all_scenario.append(best_valid_itr)
        params_for_all_scenario.append(params_traj)
        ms_s_for_all_scenario.append(ms_s_traj)
        Cs_s_for_all_scenario.append(Cs_s_traj)
        pred_ms_for_all_scenario.append(pred_ms_traj)
        pred_Cs_for_all_scenario.append(pred_Cs_traj)
        train_pred_ms_for_all_scenario.append(train_pred_ms_traj)

        print("best val error: {} at {}th iteration, Model Performance: {} | theta_dim = {}".format(last_avg_error,
                                                                                                    best_itr,
                                                                                                    model_performance,
                                                                                                    theta_dim))
        print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
        
    return best_valid_itr_for_all_scenario


In [ ]:
GetSimulationResults()

In [3]:

import site
import shutil
src = glob.glob(os.path.join(os.path.expanduser('~/'), '.ipython', 'cython', 'cylib*so'))[0]
dst = site.getsitepackages()[0]
shutil.copy(src, dst)


with dv.sync_imports():
    import cylib

NameError: name 'glob' is not defined

In [2]:
from ipyparallel import Client

rc = Client()
dview = rc[:]

with dview.sync_imports():
    import sys
    sys.path[:] = ['something']

def parallel(x):
    import sys
    return sys.path

print('Local: ', sys.path)
print()
print('Remote: ', dview.map_sync(parallel, range(1)))

importing sys on engine(s)
Local:  ['something']

Remote:  [['', '', 'D:\\Change\\Research\\StochasticPricingModelSayanMukherjee\\EchoStateNeuralStochasticVolatilityHestonModel\\experiment', 'C:\\Users\\zd26\\Anaconda3\\python37.zip', 'C:\\Users\\zd26\\Anaconda3\\DLLs', 'C:\\Users\\zd26\\Anaconda3\\lib', 'C:\\Users\\zd26\\Anaconda3', 'C:\\Users\\zd26\\Anaconda3\\lib\\site-packages', 'C:\\Users\\zd26\\Anaconda3\\lib\\site-packages\\win32', 'C:\\Users\\zd26\\Anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\Users\\zd26\\Anaconda3\\lib\\site-packages\\Pythonwin', 'C:\\Users\\zd26\\Anaconda3\\lib\\site-packages\\IPython\\extensions']]
